# LOKI Detector Diagnostics

In this notebook, we showcase two tools that can be used to investigate/debug the Loki detectors:

- `LokiBankViewer`: a 2D view of all detector panels with sliders to select straw and layer
- `InstrumentView`: a 3D view of detector panels with toggle buttons for panel selection

In [ ]:
%matplotlib widget

## Load detector bank data

We load all 9 detector banks from a Loki file:

In [ ]:
import scipp as sc
from ess import loki
import ess.loki.data  # noqa: F401
from ess.sans.types import RawDetector, SampleRun, NeXusDetectorName, BeamCenter, Filename

wf = loki.LokiWorkflow()
wf[BeamCenter] = sc.vector([0, 0, 0], unit='m')
wf[Filename[SampleRun]] = loki.data.loki_coda_file_small()

data = sc.DataGroup()
for i in range(9):
    key = f"loki_detector_{i}"
    wf[NeXusDetectorName] = key
    data[key] = wf.compute(RawDetector[SampleRun])

data.keys()

## The `LokiBankViewer`

In [ ]:
from ess.loki.diagnostics import LokiBankViewer

viewer = LokiBankViewer(data.hist())

In [ ]:
# The figures don't display well in the docs with ipympl,
# so we replace the widgets with a screenshot of the bank viewer for the docs.
import ipywidgets as ipw

file = open("../../_static/images/loki_bank_viewer.png", "rb")
viewer = ipw.Image(value=file.read(), format='png')

In [ ]:
viewer

## The `InstrumentView`

We histogram the data into an additional `event_time_offset` dimension,
which will be controlled via a slider below the instrument view.

In [ ]:
dhist = data.hist(event_time_offset=200)

In [ ]:
# Reduce number of pixels displayed to avoid size bloat in docs
dhist = dhist['pixel', ::4]['layer', ::2]

In [ ]:
from ess.loki.diagnostics import InstrumentView

InstrumentView(dhist, dim='event_time_offset')